In [1]:
pip install opendartreader

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U finance-datareader

Note: you may need to restart the kernel to use updated packages.


In [3]:
import OpenDartReader

#API Key 지정
my_api = 'c0b8eaefea99896a8137c2c8c3ce12486531fa57'

#OpenDartReader 객체 생성
dart = OpenDartReader(my_api)

### 코스피, 코스닥 종목 필터링

In [4]:
import FinanceDataReader as fdr
stock_list = fdr.StockListing('KRX') # 코스피, 코스닥, 코넥스 종목 모두 불러오기
stock_list = stock_list.loc[stock_list['Market'].isin(['KOSPI', 'KOSDAQ'])]

#stock_name_list = stock_list['Name'].tolist()
stock_name_list = stock_list.loc[~(stock_list['Name'].str.endswith(('우C','우B', '우', '홀딩스', '지주', '지주회사')) | stock_list['Name'].str.contains('스팩')), 'Name'] # 우선주, 지주사, 스팩주 제외

### 함수1: 재무정보 조회할 조건 작성

In [5]:
def get_유동자산(df):
    # 유동자산
    return (df.sj_nm == '재무상태표') & ((df.account_nm == '유동자산') | (df.account_nm == 'II. 유동자산') |
                                    (df.account_nm == 'Ⅰ.유동자산') | (df.account_nm == '유동자산 합계') | (
                                            df.account_nm == 'I.유동자산') |
                                    (df.account_nm == 'I. 유동자산') | (df.account_nm == 'l. 유동자산'))

def get_유동부채(df):
    # 유동자산
    return (df.sj_nm == '재무상태표') & ((df.account_nm == '유동부채') | (df.account_nm == 'II. 유동부채') |
                                    (df.account_nm == 'Ⅰ.유동부채') | (df.account_nm == '유동부채 합계') | (
                                            df.account_nm == 'I.유동부채') |
                                    (df.account_nm == 'I. 유동부채') | (df.account_nm == 'l. 유동부채'))


def get_부채총계(df):
    # 부채총계
    return (df.sj_nm == '재무상태표') & ((df.account_nm == '부채총계') | (df.account_nm == '부  채  총  계') |
                                    (df.account_nm == '부채 총계'))  # 부채총계


def get_자본총계(df):
    # 자본 총계
    return (df.sj_nm == '재무상태표') & \
           ((df.account_nm == '자본총계') | (df.account_nm == '반기말자본') | (df.account_nm == '3분기말자본') |
            (df.account_nm == '분기말자본') | (df.account_nm == '1분기말자본') | (df.account_nm == '자  본  총  계') |
            (df.account_nm == '기말') | (df.account_nm == '자본 총계') | (df.account_nm == '기말자본 잔액') |
            (df.account_nm == '분기말') | (df.account_nm == '반기말') | (df.account_nm == '자 본 총 계') |
            (df.account_nm == '분기말 잔액') | (df.account_nm == '반기말 잔액'))


def get_기본주당이익(df):
    # eps
    return ((df.sj_nm == '손익계산서') | (df.sj_nm == '포괄손익계산서')) & ((df.account_nm == '기본주당이익(손실)') |
                                                               (df.account_nm =='보통주 기본 및 희석주당이익(손실)')|
                                                               (df.account_nm =='기본주당순이익')|
                                                               (df.account_nm =='기본주당순이익(손실)')|
                                                               (df.account_nm =='기본주당손실')|
                                                               (df.account_nm =='기본주당손익')|
                                                               (df.account_nm =='기본주당이익(손실)')|
                                                               (df.account_nm =='계속영업기본(희석)주당손익'))


def get_매출액(df):
    # 매출액 부분
    return ((df.sj_nm == '손익계산서') | (df.sj_nm == '포괄손익계산서')) & ((df.account_nm == '매출액') |
                                                                (df.account_nm == '수익(매출액)') | (
                                                                        df.account_nm == '매출') | (
                                                                        df.account_nm == '영업수익') |
                                                                (df.account_nm == '수익') | (
                                                                        df.account_nm == '영업수익(매출액)') | (
                                                                        df.account_nm == 'Ⅰ.매출') |
                                                                (df.account_nm == 'I.매출액') | (
                                                                        df.account_nm == 'Ⅰ.매출액') | (
                                                                        df.account_nm == 'I. 매출액') |
                                                                (df.account_nm == '매출액(영업수익)'))


def get_매출총이익(df):
    # 매출총이익
    return ((df.sj_nm == '손익계산서') | (df.sj_nm == '포괄손익계산서'))& ((df.account_nm == '매출총이익') | (df.account_nm == '매출 총이익') |
    (df.account_nm == '매출총이익(손실)') | (df.account_nm == 'Ⅲ.매출총이익') |
    (df.account_nm == 'III.매출총이익') | (df.account_nm == 'Ⅲ.매출총이익') |
    (df.account_nm == 'III. 매출총이익(손실)') | (df.account_nm == 'III. 매출총이익') |
    (df.account_nm == '매출총이익(영업수익)') | (df.account_nm == 'II.재료비'))


def get_영업이익(df):
    # 영업이익
    return ((df.sj_nm == '손익계산서') | (df.sj_nm == '포괄손익계산서')) & \
           ((df.account_nm == '영업이익(손실)') | (df.account_nm == '영업이익') |
            (df.account_nm == '영업손실(이익)') | (df.account_nm == '영업손익') |
            (df.account_nm == '계속영업이익(손실)') | (df.account_nm == 'Ⅳ.영업이익') |
            (df.account_nm == 'VI.영업이익(손실)') | (df.account_nm == 'V.영업이익') | (df.account_nm == 'V. 영업이익(손실)') |
            (df.account_nm == 'IV. 영업이익') | (df.account_nm == '영업이익 (손실)') | (df.account_nm == '영업손실'))


def get_당기순이익(df):
    # 당기순이익
    return ((df.sj_nm == '손익계산서') | (df.sj_nm == '포괄손익계산서') | (df.sj_nm == '현금흐름표')) & \
           ((df.account_nm == '당기순이익(손실)') | (df.account_nm == '당기순이익') |
            (df.account_nm == '분기순이익') | (df.account_nm == '분기순이익(손실)') | (df.account_nm == '반기순이익') |
            (df.account_nm == '반기순이익(손실)') | (df.account_nm == '연결분기순이익') | (df.account_nm == '연결반기순이익') |
            (df.account_nm == '연결당기순이익') | (df.account_nm == '연결분기(당기)순이익') | (df.account_nm == '연결반기(당기)순이익') |
            (df.account_nm == '연결분기순이익(손실)') | (df.account_nm == '당기순손익') | (df.account_nm == 'Ⅶ.당기순이익') |
            (df.account_nm == 'VIII.당기순이익(손실)') | (df.account_nm == 'XIII. 당기순이익(손실)') | (
                    df.account_nm == '반기연결순이익(손실)') |
            (df.account_nm == '연결당기순이익(손실)') | (df.account_nm == '당기의 순이익') | (df.account_nm == '분기기순이익(손실)') |
            (df.account_nm == '분기순손익') | (df.account_nm == '분기순손실') | (df.account_nm == '반기순손익') | (
                    df.account_nm == '분기연결순손실') |
            (df.account_nm == '분기연결순이익(손실)'))


# 현금흐름표 부분
def get_영업현금흐름(df):
    ## 영업활동 현금흐름
    return (df.sj_nm == '현금흐름표') & ((df.account_nm == '영업활동으로 인한 현금흐름') | (df.account_nm == '영업활동 현금흐름') |
                                    (df.account_nm == '영업활동현금흐름') | (df.account_nm == '영업활동으로인한 현금흐름') | (
                                            df.account_nm == '영업활동으로인한순현금흐름') |
                                    (df.account_nm == 'Ⅰ. 영업활동으로 인한 현금흐름') | (df.account_nm == 'I.영업활동현금흐름') | (
                                            df.account_nm == '영업활동으로인한현금흐름') |
                                    (df.account_nm == '영업활동으로 인한 순현금흐름') | (df.account_nm == '영업활동으로인한 순현금흐름') | (
                                            df.account_nm == '1.영업활동현금흐름') |
                                    (df.account_nm == 'Ⅰ.영업활동현금흐름') | (df.account_nm == '영업활동으로 인한 현금 흐름') | (
                                            df.account_nm == 'I. 영업활동현금흐름') |
                                    (df.account_nm == 'I. 영업활동으로 인한 현금흐름') | (df.account_nm == 'I.영업활동으로 인한 현금흐름') |
                                    (df.account_nm == '영업활동순현금흐름 합계') | (df.account_nm == 'Ⅰ. 영업활동현금흐름') |
                                    (df.account_nm == '영업으로부터 창출된 현금흐름') | (df.account_nm == 'Ⅰ. 영업활동으로 인한 순현금흐름'))


def get_투자현금흐름(df):
    # 투자활동으로인한 현금흐름
    return (df.sj_nm == '현금흐름표') & ((df.account_nm == '투자활동으로 인한 현금흐름') | (df.account_nm == '투자활동 현금흐름') |
                                    (df.account_nm == '투자활동현금흐름') | (df.account_nm == '투자활동으로인한 현금흐름') | (
                                            df.account_nm == '투자활동으로인한순현금흐름') |
                                    (df.account_nm == 'Ⅱ. 투자활동으로 인한 현금흐름') | (df.account_nm == 'II.투자활동현금흐름') | (
                                            df.account_nm == '투자활동으로인한현금흐름') |
                                    (df.account_nm == '투자활동으로 인한 순현금흐름') | (df.account_nm == '투자활동으로인한 순현금흐름') | (
                                            df.account_nm == '2.투자활동현금흐름') |
                                    (df.account_nm == 'Ⅱ.투자활동현금흐름') | (df.account_nm == 'Ⅱ. 투자활동현금흐름') | (
                                            df.account_nm == 'II. 투자활동으로 인한 현금흐름') |
                                    (df.account_nm == 'II.투자활동으로 인한 현금흐름') | (df.account_nm == 'II. 투자활동현금흐름') |
                                    (df.account_nm == '투자활동순현금흐름 합계') | (df.account_nm == '투자활동으로부터의 순현금유입(유출)') | (
                                            df.account_nm == 'Ⅱ. 투자활동으로 인한 순현금흐름'))


def get_자산총계(df):
    # 자산총계
    return (df.sj_nm == '재무상태표') & ((df.account_nm == '자산총계') | (df.account_nm == '자  산  총  계') |
                                    (df.account_nm == '자산 총계') | (df.account_nm == '자본과부채총계'))  # 자산총계


def get_매출원가(df):
    # 매출 원가
    return ((df.sj_nm == '손익계산서') | (df.sj_nm == '포괄손익계산서')) & (df.account_nm == '매출원가')


def get_법인세비용차감전순이익(df):
    # 순이익
    return ((df.sj_nm == '손익계산서') | (df.sj_nm == '포괄손익계산서')) & (df.account_nm == '법인세비용차감전순이익(손실)')


def __check_error(self, report, condition, c):

    try:
        return report.loc[condition, c].iloc[0]

    except IndexError:
        return np.nan
    except ValueError:
        return np.nan

### 함수2: 재무정보 가져오기

In [6]:
import numpy as np
import pandas as pd

def find_financial_indicators(stock_name, year, indicators) :
    test_report = dart.finstate_all(stock_name, year)
    if test_report.empty:
        report = dart.finstate_all(stock_name, year, fs_div = 'OFS')
    else:
        report = test_report
    
    if report.empty :
        # 리포트가 없으면 당기, 전기, 전전기 값 모두 제거
        data = [[stock_name, year] + [np.nan]*len(indicators)]
        data = [[stock_name, year - 1] + [np.nan]*len(indicators)]
        data = [[stock_name, year - 2] + [np.nan]*len(indicators)]
        return pd.DataFrame(data, columns = ['기업', '연도'] + indicators)

    else:
        data = []
        for y, c in zip([year, year-1, year-2], ['thstrm_amount', 'frmtrm_amount', 'bfefrmtrm_amount']):
            record = [stock_name, y]
            for indic in indicators:
                if indic == '유동자산':
                    c1 = get_유동자산(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '유동부채':
                    c1 = get_유동부채(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '부채총계':
                    c1 = get_부채총계(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '자본총계':
                    c1 = get_자본총계(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '기본주당이익':
                    c1 = get_기본주당이익(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '매출액':
                    c1 = get_매출액(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '매출총이익':
                    c1 = get_매출총이익(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '영업이익':
                    c1 = get_영업이익(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '당기순이익':
                    c1 = get_당기순이익(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '영업현금흐름':
                    c1 = get_영업현금흐름(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '투자현금흐름':
                    c1 = get_투자현금흐름(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '자산총계':
                    c1 = get_자산총계(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:
                        value = report.loc[c1, c].iloc[0]
                elif indic == '매출원가':
                    c1 = get_매출원가(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                elif indic == '법인세비용차감전순이익':
                    c1 = get_법인세비용차감전순이익(report)
                    if sum(c1) <= 0:
                        value = np.nan
                    else:    
                        value = report.loc[c1, c].iloc[0]
                else :
                    value = np.nan
                   
                    
                record.append(value)

            data.append(record)
    return pd.DataFrame(data, columns = ['기업', '연도'] + indicators)

### 모든 종목 수집

In [7]:
import time
indicators = ['유동자산', '유동부채', '부채총계', '자본총계', '기본주당이익', '매출액','매출총이익','영업이익','당기순이익','영업현금흐름'
             ,'투자현금흐름','자산총계','매출원가','법인세비용차감전순이익']
data = pd.DataFrame()

In [8]:
for idx, stock_name in enumerate(stock_name_list):
    print(idx + 1, '/', len(stock_name_list)) #현재까지 진행된 상황 출력
    for year in [2019, 2022]:
        try:
            result = find_financial_indicators(stock_name, year, indicators)
        except:
            pass
        data = pd.concat([data, result], axis = 0, ignore_index = True)
        time.sleep(0.5) #api 분당 1000회 넘으면 제지당해서 넣은 함수

1 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
3 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
4 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
5 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
6 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
7 / 2277
8 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
47 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
48 / 2277
49 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
50 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
51 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
52 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
53 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message':

92 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
93 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
94 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
95 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
96 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
97 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
98 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message':

139 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
140 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
141 / 2277
142 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
143 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
144 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
145 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_di

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
190 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
191 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
192 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
193 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
194 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
195 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (

...

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2102 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2103 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2104 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

2105 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2106 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2107 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

2144 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2145 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2146 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

2147 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'


reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2179 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2180 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
2181 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2182 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2183 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2184 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div

2217 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

2218 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2219 / 2277
2220 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2221 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이

2261 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2262 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2263 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
2264 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='OFS' (사업보고서, 별도(개별)제무제표)'
2265 / 2277
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
2266 / 2277
reprt_code='11011', 

### 숫자로 변환

In [34]:
# 지표들이 문자로 되어 있어 숫자로 바꾸어 주기
def str_to_float(value):
    if type(value) == float:
        return value
    elif value == '-':
        return 0
    elif value == '':
        return 0
    else:
        return float(value)

for i in indicators:
    data[i] = data[i].apply(str_to_float)

In [36]:
data.to_csv('재무제표.csv', index=False, encoding='UTF-8-sig')

In [ ]:
test = dart.finstate_all('TCC스틸', 2022, fs_div = 'CFS')
test.loc[test['sj_nm'] == '포괄손익계산서']